In [ ]:
# Função para conversão de byte code para inteiro
def toInteger(octaBin):
    intNumber = 0
    octaBin = list(octaBin)
    for i in range(len(octaBin)):
        octaBin[i] = int(octaBin[i])
    intNumber += pow((octaBin[0] * 2),7)
    intNumber += pow((octaBin[1] * 2),6)
    intNumber += pow((octaBin[2] * 2),5)
    intNumber += pow((octaBin[3] * 2),4)
    intNumber += pow((octaBin[4] * 2),3)
    intNumber += pow((octaBin[5] * 2),2)
    intNumber += pow((octaBin[6] * 2),1)
    intNumber += octaBin[7]
    return(intNumber)

In [ ]:
# Importacao da biblioteca
from PIL import Image
# Inicializacao das variáveis
im = Image.open(" ---ENDEREÇO DA PASTA ONDE SUA IMAGEM ESTÁ SALVO E A IMAGEM EM SI COM EXTENSÃO--- ")
mensagem = " ---SEU SEGREDO AQUI--- "
coordX = 0
coordY = 0
maxX = im.size[0]
maxY = im.size[1]

print(maxX,maxY)

# Iterando cada letra da mensagem a ser ocultada, disparando a rotina abaixo
for i in range(0, len(mensagem)):
    # transformando a letra em bits (tabela ASCII)
    octaChar = '{0:08b}'.format(ord(mensagem[i]),'b')
    octaChar = list(octaChar)
    # variáveis de controle
    k = 0
    charCounter = 0
    # Iterando cada bit do byte ASCII
    for j in range(0,7,3):
        # Condicional se o eixo X já chegou no índice máximo, caso sim, o "ponteiro"
        # volta tudo para a esquerda e desce uma linha no eixo Y
        # (mesmo sentido de leitura de livros, por exemplo)
        if (coordX+k == maxX):
            coordY += 1
            coordX = 0
        # Coletando os níveis RGB do determinado pixel
        r, g, b = im.getpixel((coordX+k,coordY))
        # Convertendo de inteiro para binário (1 byte)
        octaR = '{0:08b}'.format(r,'b')
        octaG = '{0:08b}'.format(g,'b')
        octaB = '{0:08b}'.format(b,'b')
        # Convertendo o bit menos significativo da cor Vermelha
        octaR = list(octaR)
        octaR[-1] = octaChar[charCounter]
        octaR = ''.join(octaR)
        # Convertendo o bit menos significativo da cor Verde
        octaG = list(octaG)
        octaG[-1] = octaChar[charCounter+1]
        octaG = ''.join(octaG)
        octaB = list(octaB)
        try:
            # Convertendo o bit menos significativo da cor Azul
            octaB[-1] = octaChar[charCounter+2]
        except:
            # Aplicando a flag de final de mensagem
            if (i >= len(mensagem)-1):
                octaB[-1] = '1'
            # Aplicando a flag de continuação da mensagem
            else:
                octaB[-1] = '0'
        octaB = ''.join(octaB)
        # Atualizando os níveis das cores RGB da foto
        im.putpixel((coordX+k,coordY),(toInteger(octaR),toInteger(octaG),toInteger(octaB)))
        # Incrementando as variáveis de controle
        charCounter += 3
        k += 1
    coordX += 3
    


# Salvando a foto manipulada com a mensagem ocultada
im.save(" ---ENDEREÇO PARA ONDE SALVAR A IMAGEM MESCLADA--- ", quality=100, compress_level=0, subsampling=0,format='TIFF')

In [ ]:
# Importacao da biblioteca
from PIL import Image
# Inicializacao das variáveis
im = Image.open("---ENDEREÇO DE ONDE A IMAGEM MESCLADA FOI SALVA---")
segredo = ''
coordX = 0
coordY = 0
maxX = im.size[0]
maxY = im.size[1]
# Variável de Controle
controle = True
# Varre os bits até encontrar o bit 1 no 9o bit
while controle != False:
    byteChar = []
    # bit 1, 4 e 7
    r, g, b = im.getpixel((coordX,coordY))    
    octaR = '{0:08b}'.format(r,'b')[-1]
    byteChar.append(octaR)
    octaG = '{0:08b}'.format(g,'b')[-1]
    byteChar.append(octaG)
    octaB = '{0:08b}'.format(b,'b')[-1]
    byteChar.append(octaB)
    # bit 2, 5 e 8
    r, g, b = im.getpixel((coordX+1,coordY))    
    octaR = '{0:08b}'.format(r,'b')[-1]
    byteChar.append(octaR)
    octaG = '{0:08b}'.format(g,'b')[-1]
    byteChar.append(octaG)
    octaB = '{0:08b}'.format(b,'b')[-1]
    byteChar.append(octaB)  
    # Verifica se o ponteiro chegou ao limite do eixo X, caso sim
    # volta o X todo para a direita e desce uma linha no eixo Y
    if (coordX+2 >= maxX):
        coordY += 1
        coordX = 0
    # bit 3, 6 e 9
    r, g, b = im.getpixel((coordX+2,coordY))    
    octaR = '{0:08b}'.format(r,'b')[-1]
    byteChar.append(octaR)
    octaG = '{0:08b}'.format(g,'b')[-1]
    byteChar.append(octaG)
    octaB = '{0:08b}'.format(b,'b')[-1]
    # Condicional de parada de leitura dos bits. 1 interrompe a leitura, 0 continua a leitura
    if (octaB == '0'):      
        byteChar.append(octaB)
        byteCodeChar = ''.join(byteChar)
        segredo += chr(toInteger(byteCodeChar))
    if (octaB == '1'):
        byteCodeChar = ''.join(byteChar)
        segredo += chr(toInteger(byteCodeChar))
        controle = False
    # incrementa a variável de controle
    coordX += 3
# imprime a mensagem
print('A mensagem é:\n'+segredo)
            
